In [10]:
from heapq import heappush, heappop
import numpy as np
import sys


class puzzle(object):

    def __init__(self, node, level):
        self.state = node
        self.level = level

    def __hash__(self):
        return hash(np.array_str(self.state.ravel()))

    def __eq__(self, other):
        return np.array_equal(other.state, self.state)

    def __gt__(self, other):
        return self.level > other.level

    def __lt__(self, other):
        return self.level < other.level

    def pretty_print(self):
        return np.array_str(self.state.ravel())

    def solvable(self):
        return len([(a, b) for i, a in enumerate(self.state.ravel()) for b in self.state.ravel()[i:] if a > b & b != 0]) % 2 == 0

    def get_children(self):
        child_list = set()
        dim = self.state.shape[0]
        i, j = map(int, np.where(self.state == 0))
        # print i,j
        if (j > 0):
            child = self.state.copy()
            child[i, j] = self.state[i, j-1]
            child[i, j-1] = 0
            p = puzzle(child, self.level+1)
            child_list.add(p)
        if (j < dim-1):
            child = self.state.copy()
            child[i, j] = self.state[i, j+1]
            child[i, j+1] = 0
            p = puzzle(child, self.level+1)
            child_list.add(p)
        if (i > 0):
            child = self.state.copy()
            child[i, j] = self.state[i-1, j]
            child[i-1, j] = 0
            p = puzzle(child, self.level+1)
            child_list.add(p)
        if (i < dim-1):
            child = self.state.copy()
            child[i, j] = self.state[i+1, j]
            child[i+1, j] = 0
            p = puzzle(child, self.level+1)
            child_list.add(p)
        return child_list


In [9]:
def get_random(dim):
    """
    It creates a random puzzle, and if it's not solvable, it creates another one until it finds one that
    is solvable

    :param dim: The dimension of the puzzle
    :return: A random puzzle that is solvable.
    """
    while True:
        random_start = puzzle(np.random.permutation(
            np.arange(9)).reshape(dim, dim), 0)
        if random_start.solvable():
            break
    return random_start

In [62]:
dim=3

goal=puzzle(np.insert(np.arange(1,dim*dim),dim*dim-1,0).reshape(dim,dim),0)

easy_start=puzzle(np.insert(np.arange(1,dim*dim),6,0).reshape(dim,dim),0)

hard = np.array([[1, 2, 3], [0, 4, 5], [6, 7, 8]])
hard_puzzle = puzzle(hard, 0)

random_puzzle = get_random(dim)

In [ ]:
print('nodo objetivo : ')
print(goal.state)
print('nodo inicial : ')
print(random_puzzle.state)
print('hijos : ')
for p in random_puzzle.get_children():
    print('Estado padre, nivel : {0}'.format(p.level))
    print(p.state)
    for c in p.get_children():
        print('Estado hijo, nivel : {0}'.format(c.level))
        print(c.state)

In [46]:
from collections import deque

class bfs_puzzle():
    
    def __init__(self,start):
        self.state=start
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=deque() # queue
        frontier.append(self.state)
        while frontier:
            node=frontier.popleft() # FIFO
            if self.is_goal(node,goal):
                return parents
            visited.add(node)
            for child in self.get_children(node):
                if child not in visited and child not in frontier:
                    parents.update({child:node})
                    frontier.append(child)
        return parents
    
class ids_puzzle():
    
    def __init__(self,start):
        self.state=start
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal,bound):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=deque() # stack
        frontier.append(self.state)
        while frontier:
            node=frontier.pop() # LIFO
            if self.is_goal(node,goal):
                return parents
            visited.add(node)
            if node.level<bound:
                for child in self.get_children(node):
                    if child not in visited and child not in frontier:
                        parents.update({child:node})
                        frontier.append(child)
        return parents
    

In [18]:
class bestfirst_puzzle():
    
    def __init__(self,start,heuristic):
        self.state=start
        self.heuristic=heuristic
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=[]
        fun=self.heuristic(self.state,goal)
        heappush(frontier,(fun,self.state))
        while frontier:
            node=heappop(frontier)[1]
            if self.is_goal(node,goal):
                return parents,len(frontier),node.level
            visited.add(node)
            for child in self.get_children(node):
                if child not in visited:
                    parents.update({child:node})
                    fun=self.heuristic(child,goal)
                    heappush(frontier,(fun,child))
        return parents,len(frontier),node.level

class astar_puzzle():
    
    def __init__(self,start,heuristic):
        self.state=start
        self.heuristic=heuristic
            
    def is_goal(self,node,goal):
        return node==goal
    
    def get_children(self,node):
        return node.get_children()
    
    def search(self,goal):
        parents={}
        visited=set()
        parents.update({self.state:None})
        frontier=[]
        fun=self.heuristic(self.state,goal)+self.state.level
        heappush(frontier,(fun,self.state))
        while frontier:
            node=heappop(frontier)[1]
            if self.is_goal(node,goal):
                return parents,len(frontier),node.level
            visited.add(node)
            for child in self.get_children(node):
                if child not in visited:
                    parents.update({child:node})
                    fun=self.heuristic(child,goal)+child.level
                    heappush(frontier,(fun,child))
        return parents,len(frontier),node.level

In [74]:
def get_path(search_results,goal):
    path=[key  for (key, value) in search_results.items() if key == goal]
    node=goal
    while search_results[node] is not None:
        parent=search_results[node]
        path.append(parent)
        node=parent
    return path

### Tiempo

* BFS

In [108]:
from time import time

def time_decorator(func):
    def wrapper(*args, **kwargs):
        lista_tiempo = []
        veces = 50
        for i in range(veces):
            time_start = time()
            a = func(*args, **kwargs)
            time_end = time()
            lista_tiempo.append(time_end - time_start)
        print('Tiempo medio de ejecución: ', sum(lista_tiempo)/veces)
        if isinstance(a, int):
            print("cota maxima: ", a)
    return wrapper

In [106]:
@time_decorator
def run_bfs():
    bfs_tree = bfs_puzzle(hard_puzzle).search(goal)
    return bfs_tree

@time_decorator
def run_ids():
    bound=10
    ids_tree =ids_puzzle(easy_start).search(goal,bound)
    return ids_tree

@time_decorator
def run_ids_bounds():
    ids_tree={}
    c=1
    while goal not in ids_tree:
        ids_tree=ids_puzzle(random_puzzle).search(goal,bound=c)
        c=c+1
    return c

In [ ]:
run_bfs()

* IDS

In [107]:
run_ids_bounds()


Tiempo medio de ejecución:  142.9375433921814
cota maxima:  32


# Heuristicas

In [109]:
def manhattan(node,goal):
    dim=node.state.shape[0]
    diff_mat=np.zeros((dim,dim))
    for i in range(dim):
        for j in range(dim):
            if goal.state[i,j]!=0:
                u,v=map(int,np.where(node.state==goal.state[i,j]))
                diff_mat[i,j]=abs(i-u)+abs(j-v)
    return int(diff_mat.sum())

def hamming(node,goal):
    dim=node.state.shape[0]
    diff_mat=np.zeros((dim,dim))
    diff_mat=node.state!=goal.state
    dist=np.sum(diff_mat.astype(int).ravel())
    if dist>0:
        dist=dist-1
    return dist

* Best First

##### Hamming

In [118]:
@time_decorator
def run_bf_hamming():
    bfs_tree = bestfirst_puzzle(hard_puzzle, hamming).search(goal)
    return bfs_tree

In [119]:
run_bf_hamming()

Tiempo medio de ejecución:  0.125242018699646


#### Manhattan

In [125]:
@time_decorator
def run_bf_manhattan():
    bfs_tree = bestfirst_puzzle(random_puzzle, manhattan).search(goal)
    return bfs_tree

In [126]:
run_bf_manhattan()

Tiempo medio de ejecución:  0.051157259941101076


* A*

##### Hamming

In [140]:
@time_decorator
def run_astar_hamming():
    bfs_tree = astar_puzzle(random_puzzle, hamming).search(goal)
    return bfs_tree

In [141]:
run_astar_hamming()

Tiempo medio de ejecución:  4.191729116439819


##### Manhatttan

In [146]:
@time_decorator
def run_astar_manhattan():
    bfs_tree = astar_puzzle(random_puzzle, manhattan).search(goal)
    return bfs_tree

In [147]:
run_astar_manhattan()

Tiempo medio de ejecución:  0.309868803024292


### Memoria

* BFS

In [97]:
bfs_tree = bfs_puzzle(hard_puzzle).search(goal)
path = get_path(bfs_tree, goal)
print(len(path))
# for p in path:
#     print('-------------------------')
#     print('nivel : {0}'.format(p.level))
#     print(p.state)

print('BFS memoria : {0:.2f} [kB]'.format(sys.getsizeof(bfs_tree)/1024))


16
BFS memoria : 576.09 [kB]


* IDS

In [113]:
ids_tree={}
c=1
while goal not in ids_tree:
    ids_tree=ids_puzzle(easy_start).search(goal,bound=c)
    c=c+1
print('cota :{0}, memoria : {1:.2f} [kB]'.format(c,sys.getsizeof(ids_tree)/1024))


cota :3, memoria : 0.35 [kB]


## Best First
* Hamming

In [120]:
solver = bestfirst_puzzle(random_puzzle, hamming)
bf_tree, k, h = solver.search(goal)
path = get_path(bf_tree, goal)

# print(bf_tree[goal].level)
# for p in get_path(bf_tree, goal):
#     print('-------------------------')
#     print('nivel : {0}'.format(p.level))
#     print(p.state)

print('Best-First memoria (Hamming): {0:.2f} [kB]'.format(sys.getsizeof(bf_tree)/1024))


Best-First memoria (Hamming): 36.09 [kB]


* Manhattan

In [129]:
solver = bestfirst_puzzle(random_puzzle, manhattan)
bf_tree, k, h = solver.search(goal)
path = get_path(bf_tree, goal)

# print(bf_tree[goal].level)
# for p in get_path(bf_tree, goal):
#     print('-------------------------')
#     print('nivel : {0}'.format(p.level))
#     print(p.state)

print('Best-First memoria (Manhattan): {0:.2f} [kB]'.format(sys.getsizeof(bf_tree)/1024))

Best-First memoria (Manhattan): 4.59 [kB]


### A*
* Hamming

In [132]:
astar_solver=astar_puzzle(random_puzzle,hamming)
astar_tree,k,h=astar_solver.search(goal)
path=get_path(astar_tree,goal)
print('A-Star Manhattan memoria : {0:.2f} [kB]'.format(sys.getsizeof(astar_tree)/1024))

A-Star Manhattan memoria : 288.09 [kB]


* Manhattan

In [135]:
astar_solver=astar_puzzle(random_puzzle,manhattan)
astar_tree,k,h=astar_solver.search(goal)
path=get_path(astar_tree,goal)

print('A-Star Manhattan memoria : {0:.2f} [kB]'.format(sys.getsizeof(astar_tree)/1024))

A-Star Manhattan memoria : 18.09 [kB]
